In [1]:
import os
import re
import pandas as pd
from bs4 import BeautifulSoup
# エンコーディング自動検出用
from charset_normalizer import from_path

In [2]:
# HTMLファイルが格納されているディレクトリのパス
directory = "www.geocities.co.jp"

In [3]:
def extract_text_from_html(file_path):
    """
    HTMLファイルからテキストを抽出する関数
    """
    try:
        # charset-normalizerを使ってファイルを自動的に読み込む
        result = from_path(file_path).best()
        content = str(result)  # decodeの代わりにstr()で文字列に変換
        soup = BeautifulSoup(content, 'html.parser')
        text = soup.get_text(separator='\n')
        cleaned_text = re.sub(r'\n+', '\n', text)
        return cleaned_text
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None  # エラーが発生した場合、空のテキストを返す

In [4]:
def extract_text_recursive_to_df(directory, max_pages=100):
    """
    指定されたディレクトリを再帰的に検索し、HTMLファイルからテキストを抽出し、
    ディレクトリ構造をキーとしてDataFrameに格納する関数
    """
    data = []
    page_count = 0
    
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".html") or file.endswith(".htm"):
                file_path = os.path.join(root, file)
                text = "".join(extract_text_from_html(file_path))
                if text is None: continue
                # ディレクトリ構造と抽出したテキストをDataFrameに格納するためリストに追加
                data.append({"directory": root + "_".join(dirs), "file": file, "text": text[:500]})  # 500文字のみ格納
                
                page_count += 1
                if page_count >= max_pages:
                    print(f"Processed {max_pages} pages. Stopping...")
                    return pd.DataFrame(data)  # DataFrameに変換して返す
    
    return pd.DataFrame(data)

In [5]:
# 実行
df = extract_text_recursive_to_df(directory)

# データフレームを表示
print(df.head())

Processed 100 pages. Stopping...
                                           directory           file  \
0  www.geocities.co.jp/Playtown-Domino/6517gagaga...   geobook.html   
1  www.geocities.co.jp/Playtown-Domino/6517gagaga...  seltutei.html   
2  www.geocities.co.jp/Playtown-Domino/6517gagaga...    ninjya.html   
3  www.geocities.co.jp/Playtown-Domino/6517gagaga...     index.html   
4  www.geocities.co.jp/Playtown-Domino/6517gagaga...  geodiary.html   

                                                text  
0  \nGGGゲストブック\n僕のゲストブックです、書き込んでくださった方ありがとうございます！...  
1  \n特捜勇者\n特捜勇者\nボルフォッグ\n \n設定資料\n時代設定\n2007\n年機械...  
2  \n第1話\n第\n1\n話\n忍者王誕生！\n \n♪ジャジャジャン！\n \n時は西暦\...  
3  \nＧＧＧ奈良支部ドラえもん課\nＧＧＧ奈良支部ドラえもん課\n \n最終更新日：\n200...  
4  \n我王神地のプラモデル日記\n私、我王神地の所有しているプラモデルの数々を自慢・・・いや批...  


In [6]:
df.to_csv("extracted_texts.csv", index=False)
